In [9]:
import yaml
import torch
import torch.nn.functional as F
from train_ptuning import KoGPTConditionalGeneration
from utils import generate_next_token

In [2]:
hparams_file = 'log/hparams.yaml'
with open(hparams_file) as f:
    hparams = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [3]:
inf = KoGPTConditionalGeneration.load_from_checkpoint('./log/KoGPT2_summary-last.ckpt', hparams=hparams)

In [4]:
tokenizer = inf.tokenizer

In [5]:
SUMMARY = '<unused1>'
PTUNING = '<unused2>'
EOS = '</s>'

In [6]:
text = """수도권 지역 일일 코로나19 신규 확진자가 지난 3차 유행 이후 반년 만에 연일 600명대를 기록하면서 다음 주 수도권 지역에 새로운 사회적 거리두기 체계를 적용할 수 있을지 관심이 쏠리고 있다.

젊은 층을 중심으로 확진자가 늘고 있고 인도에서 유래한 델타 변이 바이러스가 수도권 집단감염 사례에서 발견되는 등 유행 상황이 심상치 않다.

감염병 전문가들은 상황이 나아질 때까지 새 거리두기 적용을 미뤄야 한다고 조언했다. 새 거리두기를 시행한다면 단계를 높여서 적용하거나 여름 휴가, 변이 유행 등을 대비해 방역 조처를 강화해야 한다고 강조했다.

2일 질병관리청 중앙방역대책본부에 따르면 이날 0시 기준 국내 일일 코로나19 신규 확진자는 826명이다.

지난달 30일과 이달 1일 연이틀 700명대로 집계된 데 이어 이날 800명대로 급증했다. 이는 3차 유행이 정점에 도달한 직후였던 1월7일 869명 이후 176일 만에 최대 규모다. 국내 발생 확진자 수도 같은 기간 최대 규모인 765명을 기록했다. 일주일간 하루 평균 환자 수는 635.6명으로, 이틀 연속 600명대에서 증가하고 있다.

특히 수도권 지역 유행세가 크게 증가하고 있다. 수도권 지역 국내 발생 확진자는 지난달 30일 631명을 기록한 데 이어 이달 1일 607명, 이날 619명으로 집계됐다. 연일 600명대를 기록한 건 지난 1월7일 이후 처음이다.

최근 수도권 지역에서 유행 증가세를 보이자 중앙재난안전대책본부(중대본)와 수도권 3개 시·도는 당초 1일 적용하려던 새 거리두기 체계 시행을 일주일 연기했다. 중대본과 수도권 지자체는 이번 주까지 유행 상황을 지켜본 후 다음 주 초에 논의를 거쳐 새 체계 적용 여부를 결정할 계획이다.
"""

In [7]:
text = text.replace('\n', '')
input_tokens = tokenizer.encode(PTUNING)* 10 + tokenizer.encode(text) + tokenizer.encode(SUMMARY)
input_tensor = torch.tensor(input_tokens).unsqueeze(0)

eos_id = tokenizer.encode(EOS)[0]

In [10]:
while True:
    pred = inf.model(input_tensor)
    next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.8)

    if next_token.item() == eos_id:
        break
    else:
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)],1)


In [11]:
tokenizer.decode(input_tensor[0])

'<unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2> 수도권 지역 일일 코로나19 신규 확진자가 지난 3차 유행 이후 반년 만에 연일 600명대를 기록하면서 다음 주 수도권 지역에 새로운 사회적 거리두기 체계를 적용할 수 있을지 관심이 쏠리고 있다.젊은 층을 중심으로 확진자가 늘고 있고 인도에서 유래한 델타 변이 바이러스가 수도권 집단감염 사례에서 발견되는 등 유행 상황이 심상치 않다.감염병 전문가들은 상황이 나아질 때까지 새 거리두기 적용을 미뤄야 한다고 조언했다. 새 거리두기를 시행한다면 단계를 높여서 적용하거나 여름 휴가, 변이 유행 등을 대비해 방역 조처를 강화해야 한다고 강조했다.2일 질병관리청 중앙방역대책본부에 따르면 이날 0시 기준 국내 일일 코로나19 신규 확진자는 826명이다.지난달 30일과 이달 1일 연이틀 700명대로 집계된 데 이어 이날 800명대로 급증했다. 이는 3차 유행이 정점에 도달한 직후였던 1월7일 869명 이후 176일 만에 최대 규모다. 국내 발생 확진자 수도 같은 기간 최대 규모인 765명을 기록했다. 일주일간 하루 평균 환자 수는 635.6명으로, 이틀 연속 600명대에서 증가하고 있다.특히 수도권 지역 유행세가 크게 증가하고 있다. 수도권 지역 국내 발생 확진자는 지난달 30일 631명을 기록한 데 이어 이달 1일 607명, 이날 619명으로 집계됐다. 연일 600명대를 기록한 건 지난 1월7일 이후 처음이다.최근 수도권 지역에서 유행 증가세를 보이자 중앙재난안전대책본부(중대본)와 수도권 3개 시·도는 당초 1일 적용하려던 새 거리두기 체계 시행을 일주일 연기했다. 중대본과 수도권 지자체는 이번 주까지 유행 상황을 지켜본 후 다음 주 초에 논의를 거쳐 새 체계 적용 여부를 결정할 계획이다.<unused1> 코로나19 감염병 전문가들은 수도권에서 코로나19가 연일 700명대를 기록하면서 재생산이 시작된 만큼 단계

In [12]:
tokenizer.decode(input_tensor[0]).split('<unused1>')[-1].strip()

'코로나19 감염병 전문가들은 수도권에서 코로나19가 연일 700명대를 기록하면서 재생산이 시작된 만큼 단계 상향 또는 완화 등 새로운 사회적 거리두기 체계 적용을 미뤄야 한다고 지적하고 있다.'